In [7]:
using Plots, ApproxFun
plotly();  # also works in gr;

# Laplace Equation $u_{xx} + u_{yy} = 0$, $u|_{\partial d} = \Re(e^{x+i y})$

In [20]:
d=Domain(-1..1)^2
g=Fun((x,y)->real(exp(x+im*y)),∂(d))  # boundary data
Δ=Laplacian(d)
u=[Dirichlet(d);Δ]\[g;0]
plot(u)

We solve with neumann on one edge
`ldirichlet/rdirichlet/lneumann/rneumann` commands specify left/right dirichet/neumann boundary conditions

In [21]:
dx=dy=Domain(-1..1)
d=dx*dy

x,y=Fun(∂(d))
x,y=vec(x),vec(y)

g=[-exp(x[1])*sin(-1);exp(1)*cos(y[2]);
    exp(x[3])*cos(1);exp(-1)*cos(y[4])]


Δ=Laplacian(d)

B=[I⊗lneumann(dy);rneumann(dx)⊗I;I⊗rdirichlet(dy);ldirichlet(dx)⊗I]
u=\([B;Δ],[g;0];tolerance=1E-10)

plot(u)

# Poisson equation $u_{xx} + u_{yy} = f(x,y)$

In [22]:
f=Fun((x,y)->exp(-10(x+0.2)^2-20(y-0.1)^2))  #default is (-1..1)^2
d=domain(f)
Δ=Laplacian(d)
u=[Dirichlet(d);Δ]\[0;f]
plot(u)

# Helmholtz $u_{xx} + u_{yy} + 100u=0, u|_{\partial d}=1$

In [23]:
d=Domain(-1..1)^2

Δ=Laplacian(d)

@time u=\([Dirichlet(d);Δ+100I],[ones(∂(d));0];tolerance=1E-5)
plot(u)

  1.003139 seconds (17.24 M allocations: 581.131 MB, 21.62% gc time)


In [24]:
d=Domain(-1..1)^2

Δ=Laplacian(d)

@time u=[neumann(d);Δ+100I]\[ones(4);0]
plot(u)

  0.432836 seconds (2.95 M allocations: 116.270 MB, 11.43% gc time)


# Screened Poisson  $u_{xx} + u_{yy} - 100u = 0, \partial u(\partial d) = 1$

In [25]:
d=Domain(-1..1)^2
Δ=Laplacian(d)

@time u=[neumann(d);Δ-100.0I]\[ones(4);0.0]
plot(u)

  0.299581 seconds (2.13 M allocations: 83.644 MB, 12.28% gc time)


# Convection $u_t + u_x = 0,u(x,0)=e^{-20x^2}, u(-1,t) = 0$

In [27]:
dx=Domain(-1..1);dt=Domain(0..2)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
u0=Fun(x->exp(-20x^2),dx)
@time u=\([I⊗ldirichlet(dt);ldirichlet(dx)⊗I;Dt+Dx],[u0;0;0];
            tolerance=1E-3)
plot(u)

  0.791594 seconds (3.18 M allocations: 132.471 MB, 34.34% gc time)


## Piecewise PDE $$u_t+\begin{cases} 0.5 & 0≤x≤0.5\cr                     1 & \hbox{otherwise}\end{cases} u_{xx}$$

We can solve on piecewise domains. Here is convection with two different speeds, imposing continuity at the singularities.

In [28]:
a=Fun(x -> 0 ≤ x ≤ 0.5 ? 0.5 : 1, Domain(-1..1) \ [0,0.5])
s=space(a)
dt=Interval(0,2.)
Dx=Derivative(s);Dt=Derivative(dt)
Bx=[ldirichlet(s);continuity(s,0)]
@time u=\([I⊗ldirichlet(dt);Bx⊗I;I⊗Dt+(a*Dx)⊗I],
    [Fun(x->exp(-20(x+0.5)^2),s);zeros(length(Bx));0.0];tolerance=1E-2)

plot(u)

 58.625242 seconds (222.64 M allocations: 8.413 GB, 6.61% gc time)


# Convection $u_t - x u_x = 0, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t)=0$

In [29]:
dx=Domain(-1..1);dt=Domain(0..2)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(identity,d)

#timedirichlet is [u(x,0), u(-1,t), u(1,t)]
u0 = Fun(x->exp(-20x^2),dx)
@time u=\([timedirichlet(d);Dt-x*Dx],[u0;0;0;0];tolerance=1E-3)
plot(u)

 22.457516 seconds (45.90 M allocations: 2.177 GB, 10.36% gc time)


# Convection $u_t + x u_x = 0, u(x,0)=e^{-20x^2}$

In [30]:
dx=Domain(-1..1);dt=Domain(0..2)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,y=Fun(d)
u0=Fun(x->exp(-20x^2),dx)
@time u=\([I⊗ldirichlet(dt);Dt+x*Dx],[u0;0];tolerance=1E-10)
plot(u)

  0.440868 seconds (2.07 M allocations: 105.477 MB, 10.58% gc time)


# Convection diffusion $u_t = \epsilon u_{xx} + (2+x) u_x, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t) = 0$

In [31]:
dx=Domain(-1..1);dt=Domain(0..1)
d=dx*dt
ε=.01
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(d)
V=2.0+x
u0=Fun(x->exp(-20x^2),dx)
@time u=\([timedirichlet(d);Dt-ε*Dx^2-V*Dx],[u0;0;0;0];
                    tolerance=1E-6)

plot(u)

  1.185534 seconds (4.93 M allocations: 194.077 MB, 31.84% gc time)


# Wave equation with left Dirichlet and right Neumann $u_{tt} = u_{xx}, u(x,0)=e^{-20(x-.1)^2},u_t(x,0)=0,u(-1,t)=0,u_x(1,t)=0$

In [32]:
dx=-1..1;dt=0..4
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
# need to specify both ic and its derivative
B=[I⊗ldirichlet(dt),I⊗lneumann(dt),ldirichlet(dx)⊗I,rneumann(dx)⊗I]

u0=Fun(x->exp(-20(x-.1)^2),dx)
@time u=\([B;Dt^2-Dx^2],[u0;0;0;0;0];
                    tolerance=1E-4)

plot(u)

  1.026443 seconds (6.72 M allocations: 250.110 MB, 8.67% gc time)


# Linear KdV Dirichlet $u_t + u_{xxx} = 0, u(x,0)=e^{-10x^2},u(-1,t)=u(1,t)=u_x(1,t)=0$

In [33]:
dx=Domain(-5..1);dt=Domain(0..0.1)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
B=[timedirichlet(d);rneumann(dx)⊗I]
u0=Fun(x->exp(-10x^2),dx)
@time u=\([B;Dt+Dx^3],[u0;0;0;0;0];tolerance=1E-3)

plot(u)

  7.375974 seconds (22.13 M allocations: 948.714 MB, 9.64% gc time)


## Linear KdV Neumann $u_t + u_{xxx} = 0, u(x,0)=e^{-10x^2},u_x(-1,t)=u(1,t)=u_x(1,t)=0$

In [8]:
dx=Domain(-5..1);dt=Domain(0..1)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
B=[I⊗ldirichlet(dt);neumann(dx)⊗I;rdirichlet(dx)⊗I]
u0=Fun(x->exp(-10x^2),dx)
@time u=\([B;Dt+Dx^3],[u0;0;0;0;0];tolerance=1E-4)

plot(u)

131.641061 seconds (214.59 M allocations: 7.769 GB, 3.38% gc time)


# Beamer equation $u_{tt} + u_{xxxx} = 0, u(x,0)=e^{-200(x-1/2)^2}, u_t(x,0)=u(\pm,t)=u_x(\pm 1,t) = 0$

In [9]:
dx=Domain(0..1);dt=Domain(0..0.03)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1]);
x,t=Fun(d)

u0=Fun(x->exp(-200(x-.5)^2),dx)

@time u=\(
    [timedirichlet(d);I⊗lneumann(dt);neumann(dx)⊗I;Dt^2+Dx^4],
    [u0;0;0;0;0;0;0];tolerance=1E-4)

plot(u)

 17.907741 seconds (78.44 M allocations: 2.362 GB, 6.41% gc time)


# Biharmonic equation $\Delta^2 u = f(x,y)$

In [10]:
S=WeightedJacobi(2.,2.)^2

Δ=Laplacian(S)

@time f=chop(Fun((x,y)->exp(-30(x^2+y^2)),rangespace(Δ^2)),1E-10)
@time u=\(Δ^2,f;tolerance=1E-8)
plot(u)

 11.045660 seconds (51.66 M allocations: 1.216 GB, 5.37% gc time)
 14.490786 seconds (16.39 M allocations: 595.275 MB, 3.15% gc time)


# Schrödinger Dirichlet $i \epsilon u_t + .5 \epsilon^2 u_{xx} = x^2 u, u(x,0)=e^{-25(x-.5)^2}e^{-i/(5ϵ)log(2cosh(5(x-.5)))}$

In [12]:
dx=Domain(0..1);dt=Domain(0..0.54)
d=dx*dt

ϵ=0.0256
u0=Fun(x->exp(-25*(x-.5)^2)*exp(-1.0im/(5*ϵ)*log(2cosh(5*(x-.5)))),dx)

x,t=Fun(d)
V=x^2

Dt=Derivative(d,[0,1]);Dx=Derivative(d,[1,0])

L=1im*ϵ*Dt+.5*ϵ^2*Dx^2-V

@time u=\([timedirichlet(d);L],[u0;0;0;0];tolerance=1E-2)

plot(real(u))

 24.546714 seconds (28.69 M allocations: 1.590 GB, 3.59% gc time)
